In [1]:
!nvidia-smi

Thu Mar 18 13:10:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# install KenLM
!apt-get install libboost-all-dev
!git clone https://github.com/kpu/kenlm.git
%cd kenlm/
!mkdir -p build
%cd build
!cmake ..
!make -j 4
%cd ..
!python setup.py install

# install nltk?
!pip install nltk
!python -m nltk.downloader -d /usr/local/share/nltk_data punkt
%cd ..

!git clone -q https://github.com/StuartMesham/low_resource_lm.git
%cd low_resource_lm
!git checkout d36ae34284daaeb8afa476be400172a915331636
!pip install -qr requirements.txt
!./scripts/fetch_data.sh

!pip install tokenizers

%cd /content/

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-all-dev is already the newest version (1.65.1.0ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Cloning into 'kenlm'...
remote: Enumerating objects: 13687, done.
remote: Total 13687 (delta 0), reused 0 (delta 0), pack-reused 13687
Receiving objects: 100% (13687/13687), 5.46 MiB | 10.35 MiB/s, done.
Resolving deltas: 100% (7880/7880), done.
/content/kenlm
/content/kenlm/build
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CX

In [ ]:
%%writefile /content/kenlm/build/process.py
import sys, nltk
for line in sys.stdin:
    for sentence in nltk.sent_tokenize(line):
        print(' '.join(nltk.word_tokenize(sentence)).lower())

Writing /content/kenlm/build/process.py


In [ ]:
!wc -l n-gram.arpa

5722000 n-gram.arpa


In [ ]:
 # TRAIN_SET = '/content/low_resource_lm/data/nchlt/sepedi_train.txt'
# VAL_SET = '/content/low_resource_lm/data/nchlt/sepedi_valid.txt'
# TEST_SET = '/content/low_resource_lm/data/nchlt/sepedi_test.txt'
# BPE_VOCAB_SIZE = 8000
# N_GRAM_ORDER = 6

DATASETS = [
  (
      '/content/low_resource_lm/data/nchlt/sepedi_train.txt',
      '/content/low_resource_lm/data/nchlt/sepedi_valid.txt',
      '/content/low_resource_lm/data/nchlt/sepedi_test.txt',
  ),
  # (
  #     '/content/low_resource_lm/data/nchlt/isizulu_train.txt',
  #     '/content/low_resource_lm/data/nchlt/isizulu_valid.txt',
  #     '/content/low_resource_lm/data/nchlt/isizulu_test.txt',
  # ),
  # (
  #     '/content/low_resource_lm/data/isolezwe/isizulu_train.txt',
  #     '/content/low_resource_lm/data/isolezwe/isizulu_valid.txt',
  #     '/content/low_resource_lm/data/isolezwe/isizulu_test.txt',
  # ),
]

f = open('results.csv', 'w')
f.write('train set, val set, test set, bpe vocab size, ngram order, val bpc, test bpc\n')

for TRAIN_SET, VAL_SET, TEST_SET in DATASETS:
  for BPE_VOCAB_SIZE in [2000]:#1000, 2000, 4000, 8000, 10000]:
    for N_GRAM_ORDER in [6]:#3, 4, 5, 6, 8]:
      from tokenizers import ByteLevelBPETokenizer

      tokenizer = ByteLevelBPETokenizer()
      tokenizer.train([TRAIN_SET], vocab_size=BPE_VOCAB_SIZE)
      tokenizer.save("mybpe")

      from pathlib import Path
      txt = Path(TRAIN_SET).read_text()

      output = tokenizer.encode(txt)
      outputIDs = output.ids

      outF = open("myOutFile.txt", "w")
      for id in outputIDs:
        strID = str(id)
        outF.write(strID)
        outF.write(" ")
      outF.close()

      txt = Path(VAL_SET).read_text()

      output = tokenizer.encode(txt)
      outputIDs = output.ids

      outF = open("myOutFileValid.txt", "w")
      for id in outputIDs:
        strID = str(id)
        outF.write(strID)
        outF.write(" ")
      outF.close()

      txt = Path(TEST_SET).read_text()

      output = tokenizer.encode(txt)
      outputIDs = output.ids

      outF = open("myOutFileTest.txt", "w")
      for id in outputIDs:
        strID = str(id)
        outF.write(strID)
        outF.write(" ")
      outF.close()


      # train n-grams

      !cat myOutFile.txt | python /content/kenlm/build/process.py | /content/kenlm/build/bin/lmplz -o $N_GRAM_ORDER > n-gram.arpa --discount_fallback

      !/content/kenlm/build/bin/build_binary n-gram.arpa n-gram.klm
      !wc -l n-gram.arpa


      # # report results

      # f.write('{}, {}, {}, {}, {}, '.format(TRAIN_SET, VAL_SET, TEST_SET, BPE_VOCAB_SIZE, N_GRAM_ORDER))

      # import kenlm
      # import math

      # model = kenlm.LanguageModel('n-gram.klm')

      # from pathlib import Path
      # txt = Path('myOutFileValid.txt').read_text().strip()
      # txt2 = Path(VAL_SET).read_text().strip()

      # per=model.perplexity(txt)
      # characters = len(txt2)
      # tokens = len(txt.split(" "))
      # print("VALIDATION SET RESULTS:")
      # print("The number of characters: ", characters)
      # print("The number of tokens: ", tokens)
      # print("Perplexity: ", per)

      # bpc = tokens/characters * math.log2(per)
      # print("Bits per character: ", bpc)

      # f.write(str(bpc) + ', ')

      # txt = Path('myOutFileTest.txt').read_text().strip()
      # txt2 = Path(TEST_SET).read_text().strip()

      # per=model.perplexity(txt)
      # characters = len(txt2)
      # tokens = len(txt.split(" "))
      # print("TEST SET RESULTS:")
      # print("The number of characters: ", characters)
      # print("The number of tokens: ", tokens)
      # print("Perplexity: ", per)

      # bpc = tokens/characters * math.log2(per)
      # print("Bits per character: ", bpc)

      # f.write(str(bpc) + '\n')
      # f.flush()

f.close()

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
tcmalloc: large alloc 1720696832 bytes == 0x557a70fda000 @  0x7f60f3ac21e7 0x557a6e6957a2 0x557a6e63051e 0x557a6e60f2eb 0x557a6e5fb066 0x7f60f1c5bbf7 0x557a6e5fcbaa
tcmalloc: large alloc 9177030656 bytes == 0x557ad78d6000 @  0x7f60f3ac21e7 0x557a6e6957a2 0x557a6e6847ca 0x557a6e685208 0x557a6e60f308 0x557a6e5fb066 0x7f60f1c5bbf7 0x557a6e5fcbaa
Unigram tokens 2269646 types 1856
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:22272 2:672176000 3:1260329984 4:2016528000 5:2940770048 6:4033056000
tcmalloc: large alloc 4033060864 bytes == 0x557a70ed8000 @  0x7f60f3ac21e7 0x557a6e6957a2 0x557a6e6847ca 0x557a6e685208 0x557a6e60f8d7 0x557a6e5fb066 0x7f60f1c5bbf7 0x557a6e5fcbaa
tcmalloc: large alloc 2016534528 bytes == 0x557bd481c000 @  0x7f60f3ac21e7 0x557a6e6957a2 0x557a6e6847ca 0x557a6e685208 0x557a6e60fcdd 0x557a6e5fb066 0x7f60f1c5bbf7 0x557a6e5fcbaa


In [ ]:
!git clone -q https://github.com/StuartMesham/low_resource_lm.git
%cd low_resource_lm
!pip install -qr requirements.txt
!./scripts/fetch_data.sh

!pip install tokenizers

%cd /content/
!git clone -q https://github.com/StuartMesham/word-language-model.git
%cd word-language-model/
!pip install -r requirements.txt
%cd /content/low_resource_lm/data/
!cp -r  /content/low_resource_lm/data/nchlt/ /content/word-language-model/
!cp -r  /content/low_resource_lm/data/isolezwe/ /content/word-language-model/
%cd /content/word-language-model/nchlt/
%cd /content/word-language-model/

/content/low_resource_lm
     |████████████████████████████████| 163kB 11.2MB/s 
     |████████████████████████████████| 71kB 5.4MB/s 
     |████████████████████████████████| 133kB 10.7MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
fetching nchlt data
processing: https://repo.sadilar.org/bitstream/handle/20.500.12185/330/corpora.nchlt.nso.zip
total sentences in sepedi.txt: 105552
processing: http://www.rma.nwu.ac.za/bitstream/handle/20.500.12185/321/corpora.nchlt.zu.zip
total sentences in isizulu.txt: 113072
processing: https://repo.sadilar.org/bitstream/handle/20.500.12185/314/corpora.nchlt.xh.zip
total sentences in isixhosa.txt: 83914
processing: https://repo.sadilar.org/bitstream/handle/20.500.12185/364/corpora.nchlt.ts.zip
total sentences in xitsonga.txt: 68688
processing: http://www.rma.nwu.ac.za/bits

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git pull

Already up to date.


In [ ]:
from tokenizers import ByteLevelBPETokenizer
TRAIN_SET = '/content/low_resource_lm/data/isolezwe/isizulu_train.txt'
tokenizer = ByteLevelBPETokenizer()
tokenizer.train([TRAIN_SET], vocab_size=8000)

In [ ]:
tokenizer

In [ ]:
DATASETS = [
  # (
  #     '/content/low_resource_lm/data/nchlt/sepedi_train.txt',
  #     '/content/low_resource_lm/data/nchlt/sepedi_valid.txt',
  #     '/content/low_resource_lm/data/nchlt/sepedi_test.txt',

  # ),
  # (
  #     '/content/low_resource_lm/data/nchlt/isizulu_train.txt',
  #     '/content/low_resource_lm/data/nchlt/isizulu_valid.txt',
  #     '/content/low_resource_lm/data/nchlt/isizulu_test.txt',

  # ),
  (
      '/content/low_resource_lm/data/isolezwe/isizulu_train.txt',
      '/content/low_resource_lm/data/isolezwe/isizulu_valid.txt',
      '/content/low_resource_lm/data/isolezwe/isizulu_test.txt',

  ),
]

# f = open('results.csv', 'w')
# f.write('train set, val set, test set, bpe vocab size, ngram order, val bpc, test bpc\n')

for TRAIN_SET, VAL_SET, TEST_SET in DATASETS:
  for BPE_VOCAB_SIZE in [10000]:#1000, 2000, 8000, 10000]:
    for N_GRAM_ORDER in [2]:#, 6]:
      from tokenizers import ByteLevelBPETokenizer

      tokenizer = ByteLevelBPETokenizer()
      tokenizer.train([TRAIN_SET], vocab_size=BPE_VOCAB_SIZE)
      tokenizer.save("mybpe")

      from pathlib import Path
      txt = Path(TRAIN_SET).read_text()

      output = tokenizer.encode(txt)
      outputIDs = output.ids

      outF = open("myOutFile.txt", "w")
      for id in outputIDs:
        strID = str(id)
        outF.write(strID)
        outF.write(" ")
      outF.close()

      txt = Path(VAL_SET).read_text()

      output = tokenizer.encode(txt)
      outputIDs = output.ids

      outF = open("myOutFileValid.txt", "w")
      for id in outputIDs:
        strID = str(id)
        outF.write(strID)
        outF.write(" ")
      outF.close()

      txt = Path(TEST_SET).read_text()

      output = tokenizer.encode(txt)
      outputIDs = output.ids

      outF = open("myOutFileTest.txt", "w")
      for id in outputIDs:
        strID = str(id)
        outF.write(strID)
        outF.write(" ")
      outF.close()

      !mv myOutFile.txt train.txt
      !mv myOutFileValid.txt valid.txt
      !mv myOutFileTest.txt test.txt

      txt = Path('valid.txt').read_text().strip()
      txt2 = Path(VAL_SET).read_text().strip()
      val_characters = len(txt2)
      val_tokens = len(txt.split(" "))

      txt = Path('test.txt').read_text().strip()
      txt2 = Path(TEST_SET).read_text().strip()
      test_characters = len(txt2)
      test_tokens = len(txt.split(" "))

      for EMSIZE in [500]:
        for DROPOUT in [0.3]:#, 0.5]:
          for BATCH_SIZE in [32]:
            for N_LAYERS in [2]:
              !python main.py \
              --cuda \
              --epochs 0 \
              --data "./" \
              --nlayers $N_LAYERS \
              --emsize $EMSIZE \
              --nhid $EMSIZE \
              --dropout $DROPOUT \
              --batch_size $BATCH_SIZE \
              --val_characters $val_characters \
              --val_tokens $val_tokens \
              --test_characters $test_characters \
              --test_tokens $test_tokens \
              --train_set_path $TRAIN_SET \
              --bpe_vocab_size $BPE_VOCAB_SIZE \
              --norder $N_GRAM_ORDER \
              --log-interval 300 \
              --csv ff_results.csv


5688354
Traceback (most recent call last):
  File "main.py", line 296, in <module>
    with open(args.save, 'rb') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'model.pt'


In [ ]:
!python main.py --cuda --epochs 10 --model FeedForward --data nchlt/

-------------------------------------------------------------
Training tokenizer on: nchlt/isizulu_/train.txt
-------------------------------------------------------------
Encoding dataset at: nchlt/isizulu_
-------------------------------------------------------------
Traceback (most recent call last):
  File "main.py", line 81, in <module>
    pad_id = corpus.dictionary.word2idx['<eos>']
KeyError: '<eos>'


In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 2 --dropout 0.65 --model FeedForward --data ../data/autshumato/

Traceback (most recent call last):
  File "main.py", line 68, in <module>
    corpus = data.Corpus(args.data)
  File "/content/word-language-model/data.py", line 23, in __init__
    self.train = self.tokenize(os.path.join(path, 'train.txt'))
  File "/content/word-language-model/data.py", line 29, in tokenize
    assert os.path.exists(path)
AssertionError


In [ ]:
!python main.py --cuda --epochs 30 --emsize 1500 --nhid 2000 --nlayers 2 --dropout 0.65 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 30 --emsize 1500 --nhid 1500 --nlayers 2 --dropout 0.35 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 30 --emsize 2500 --nhid 1500 --nlayers 2 --dropout 0.35 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 30 --emsize 1500 --nhid 1500 --nlayers 2 --dropout 0.25 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 3 --dropout 0.65 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.65 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 2 --dropout 0.75 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 2 --dropout 0.55 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 2 --dropout 0.45 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 100 --emsize 1500 --nhid 1500 --nlayers 2 --dropout 0.65 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 3000 --nhid 1500 --nlayers 2 --dropout 0.65 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 750 --nlayers 2 --dropout 0.65 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 2 --dropout 0.65 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --emsize 650 --nhid 650 --dropout 0.5 --epochs 40 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --emsize 650 --nhid 650 --dropout 0.5 --epochs 40 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.65 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.65 --bptt 70 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.65 --bptt 70 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.65 --bptt 140 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.60 --bptt 70 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.70 --bptt 70 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.30 --bptt 70 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 3 --dropout 0.65 --bptt 70 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 4 --dropout 0.65 --bptt 70 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.70 --bptt 140 --tied --model FeedForward --data ../data/autshumato/


In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 4 --dropout 0.55 --bptt 70 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 4 --dropout 0.45 --bptt 70 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 4 --dropout 0.35 --bptt 70 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 4 --dropout 0.75 --bptt 70 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.70 --bptt 280 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.70 --bptt 560 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 25 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.70 --bptt 280 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 75 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.70 --bptt 280 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.75 --bptt 280 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.65 --bptt 280 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 30 --emsize 1250 --nhid 1250 --nlayers 1 --dropout 0.70 --bptt 280 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1750 --nhid 1750 --nlayers 1 --dropout 0.70 --bptt 280 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 30 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.70 --bptt 280 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.70 --bptt 280 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 75 --emsize 1500 --nhid 1500 --nlayers 1 --dropout 0.70 --bptt 280 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2500 --nhid 2500 --nlayers 1 --dropout 0.70 --bptt 280 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1500 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 1500 --nlayers 1 --dropout 0.70 --bptt 280 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 1000 --nlayers 1 --dropout 0.70 --bptt 280 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 3000 --nhid 3000 --nlayers 1 --dropout 0.70 --tied --bptt 280 --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 30 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 40 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 10 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 30 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 40 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 10 --norder 1 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 10 --norder 3 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 10 --norder 4 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 10 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 10 --norder 6 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 10 --norder 2 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --emsize 650 --nhid 650 --dropout 0.5 --epochs 40 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --emsize 650 --nhid 650 --dropout 0.5 --epochs 40 --norder 3 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --emsize 650 --nhid 650 --dropout 0.5 --epochs 40 --norder 4 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --emsize 650 --nhid 650 --dropout 0.5 --epochs 40 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 10 --norder 2 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 10 --norder 1 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 10 --norder 3 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 1000 --nhid 1000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 20 --norder 4 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 10 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.60 --bptt 280 --batch_size 10 --lr 10 --norder 4 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 2 --dropout 0.60 --bptt 280 --batch_size 10 --lr 10 --norder 4 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 40 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.50 --bptt 280 --batch_size 10 --lr 20 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 40 --emsize 1000 --nhid 1000 --nlayers 1 --dropout 0.50 --bptt 280 --batch_size 10 --lr 20 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 40 --emsize 1000 --nhid 1000 --nlayers 1 --dropout 0.60 --bptt 280 --batch_size 10 --lr 20 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 40 --emsize 1000 --nhid 1000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 20 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --emsize 650 --nhid 650 --dropout 0.5 --epochs 40 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --emsize 1500 --nhid 1500 --dropout 0.65 --epochs 40 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 40 --emsize 1000 --nhid 1000 --nlayers 1 --dropout 0.80 --bptt 280 --batch_size 10 --lr 20 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 40 --emsize 750 --nhid 750 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 20 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 40 --emsize 1250 --nhid 1250 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 20 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 10 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 20 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 50 --emsize 2000 --nhid 2000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 10 --norder 2 --tied --model FeedForward --data ./autshumato/

In [ ]:
!python main.py --cuda --epochs 40 --emsize 1000 --nhid 1000 --nlayers 1 --dropout 0.50 --bptt 280 --batch_size 10 --lr 30 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 40 --emsize 1000 --nhid 1000 --nlayers 1 --dropout 0.70 --bptt 280 --batch_size 10 --lr 30 --norder 5 --tied --model FeedForward --data ../data/autshumato/

In [ ]:
!python main.py --cuda --epochs 40 --emsize 650 --nhid 650 --nlayers 1 --dropout 0.4 --bptt 280 --batch_size 10 --lr 20 --norder 5 --tied --model FeedForward --data ../data/autshumato/